In [1]:
from dotenv import load_dotenv, find_dotenv
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool
# import nest_asyncio

# nest_asyncio.apply()

_ = load_dotenv(find_dotenv())

In [2]:
all_inference_models = !curl -s https://huggingface.co/api/models | jq ".[].id"

In [ ]:
from huggingface_hub import model_info


'warm'

In [14]:

info = model_info("Qwen/Qwen2.5-VL-72B-Instruct", expand="inference")
info.inference


'warm'

In [2]:
from smolagents import Tool

class SaveContent(Tool):
    name = "SaveContent"
    description = "Saves content to a file."
    inputs = {
        "content": {
            "type": "string",
            "description": "The content to save."
        },
        "filename": {
            "type": "string",
            "description": "The filename."
        }
    }
    output_type = "string"

    def forward(self, content: str, filename: str ):
        with open(filename, "w") as f:
            f.write(content)
        return f"Content saved to {filename}"



In [17]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), SaveContent()], 
                  model=InferenceClientModel(bill_to="VitalNest",max_tokens=150000),
                  additional_authorized_imports=["os", "requests", "bs4"],
                 max_steps=10, planning_interval=4,
                 )

In [ ]:
agent.run(f"""Use a headless browser agent to navigate to this page: https://www.hdfcergo.com/download/policy-wordings.
Retrieve all the policy wording documents under health category.""")

In [1]:
import torch
from PIL import Image
from transformers.utils.import_utils import is_flash_attn_2_available

from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor


In [2]:

model = ColQwen2_5.from_pretrained(
    "vidore/colqwen2.5-v0.2",
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",  # or "mps" if on Apple Silicon
    attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
).eval()
processor = ColQwen2_5_Processor.from_pretrained("vidore/colqwen2.5-v0.2", use_fast=True)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:

# Your inputs
images = [
    Image.new("RGB", (128, 128), color="white"),
    Image.new("RGB", (128, 128), color="black"),
]
queries = [
    "Is attention really all you need?",
    "What is the amount of bananas farmed in Salvador?",
]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

# Forward pass
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)
